In [22]:
%matplotlib inline 
import pandas as pd
import numpy as np
import re
import pprint
import gensim.corpora as corpora
import gensim.models as models

In [3]:
ls

2014 annuals
Chevron Annual Report Wordnet.ipynb
Untitled.ipynb
annualreports.mdl
chevron2014.txt
clusteroutput.model
corpus.mm
corpus.mm.index
foster-wheeler-2013-summary-annual-report.pdf
multiple oil and gas.ipynb
multiple2014.txt
multiple_output.bin
output.bin
pdf.pikl
rawtut.ipynb
sentences as docs.ipynb
text8*


In [4]:
### chevron - added conocophilips, exxon mobil, 
### independent oil and gas: imperial oil, Abraxas, chesapeake
### comstock resources, jones energy, US Energy Corp, ULTRA petroleum
### all oil & gas refining: alon resource partners, atlas resource,
### calumet specialty products partners, CVR Energy, eagle rock energy,
### hess corp, philips 66 partners, valero energy
!head 2014\ annuals

 
UNITED STATES SECURITIES AND EXCHANGE COMMISSION
Washington, D.C. 20549
Form 10-K        
 
þ  ANNUAL REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE
SECURITIES EXCHANGE ACT OF 1934
For the fiscal year ended December 31, 2014
OR    
o  TRANSITION REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE


#### join all new breaks
the assumption is that periods demark sentences, NOT return carriages (\n)

In [5]:
tmp = open('2014 annuals', 'rb')
tmptmp = " ".join(tmp)
print tmptmp[:100]
print type(tmptmp)

 
 UNITED STATES SECURITIES AND EXCHANGE COMMISSION
 Washington, D.C. 20549
 Form 10-K        
  
 �
<type 'str'>


In [6]:
# deprecated
# tmptmp = tmptmp.replace("\n  \n", ". ").strip()
# print tmptmp[:500]

In [7]:
# deprecated
# tmptmp = re.sub( '\s+', ' ', tmptmp ).strip()
# print tmptmp[:500]

In [8]:
stoplist = set('for a of the and to in'.split())


In [9]:
tmptmp = tmptmp.replace("\n  \n", ". ").strip()
tmptmp = re.sub( '\s+', ' ', tmptmp ).strip()
tmptmp = tmptmp.replace(' .', '.')
tmptmp = tmptmp.replace(' o ', ". ")
# tmptmp[:1000].split('. ')

In [10]:
texts = [word for word in tmptmp.lower().split('. ') if word not in stoplist]

texts[:50]

['united states securities and exchange commission washington, d.c',
 '20549 form 10-k',
 '\xc3\xbe annual report pursuant to section 13 or 15(d) of the securities exchange act of 1934 for the fiscal year ended december 31, 2014 or',
 'transition report pursuant to section 13 or 15(d) of the securities exchange act of 1934 for the transition period from to commission file number 001-00368 chevron corporation (exact name of registrant as specified in its charter) delaware',
 '94-0890210',
 '6001 bollinger canyon road, san ramon, california 94583-2324 (state or other jurisdiction of incorporation or organization)',
 '(i.r.s',
 'employer identification no.)',
 '(address of principal executive offices) (zip code)',
 'registrant\xe2\x80\x99s telephone number, including area code (925) 842-1000',
 'securities registered pursuant to section 12 (b) of the act:',
 'title of each class',
 'name of each exchange on which registered common stock, par value $.75 per share',
 'new york stock exchang

In [11]:
from collections import defaultdict
frequency = defaultdict(int)
for text in texts:
    for token in text.split(" "):
        frequency[token] += 1

In [12]:
# for i in texts:
#     print [j for j in i.split(" ") if frequency[j] > 1 and j not in stoplist]
#     stop

In [13]:
texts2 = []
for line in texts:
    texts2.append([j for j in line.split(" ") if frequency[j] > 1 and j not in stoplist])

In [14]:
pprint.pprint(texts2[:10])

[['united',
  'states',
  'securities',
  'exchange',
  'commission',
  'washington,',
  'd.c'],
 ['20549', 'form', '10-k'],
 ['\xc3\xbe',
  'annual',
  'report',
  'pursuant',
  'section',
  '13',
  'or',
  '15(d)',
  'securities',
  'exchange',
  'act',
  '1934',
  'fiscal',
  'year',
  'ended',
  'december',
  '31,',
  '2014',
  'or'],
 ['transition',
  'report',
  'pursuant',
  'section',
  '13',
  'or',
  '15(d)',
  'securities',
  'exchange',
  'act',
  '1934',
  'transition',
  'period',
  'from',
  'commission',
  'file',
  'number',
  'chevron',
  'corporation',
  '(exact',
  'name',
  'registrant',
  'as',
  'specified',
  'its',
  'charter)',
  'delaware'],
 [],
 ['6001',
  'bollinger',
  'canyon',
  'road,',
  'san',
  'ramon,',
  'california',
  '94583-2324',
  '(state',
  'or',
  'other',
  'jurisdiction',
  'incorporation',
  'or',
  'organization)'],
 ['(i.r.s'],
 ['employer', 'identification', 'no.)'],
 ['(address', 'principal', 'executive', 'offices)', '(zip', 'code)'

In [15]:
dictionary = corpora.Dictionary(texts2)
dictionary.save('annualreports.mdl') # store the dictionary, for future reference
print(dictionary)


Dictionary(23756 unique tokens: [u'1,808', u'1,809', u'"greenhouse', u'considered,', u'1,803']...)


In [16]:
new_doc = "currency markets for oil are"
new_vec = dictionary.doc2bow(new_doc.lower().split())
print(new_vec) # the word "interaction" does not appear in the dictionary and is ignored
# returns [(token_id, token_count), ...] for incoming sentence

[(375, 1), (502, 1), (512, 1), (701, 1)]


In [17]:
corpus = [dictionary.doc2bow(text) for text in texts2]
# corpora.MmCorpus.serialize('/tmp/deerwester.mm', corpus) # store to disk, for later use

In [18]:
print(corpus[:5])

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1)], [(7, 1), (8, 1), (9, 1)], [(3, 1), (4, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1), (22, 1), (23, 1), (24, 2), (25, 1)], [(3, 1), (4, 1), (5, 1), (11, 1), (13, 1), (15, 1), (16, 1), (17, 1), (19, 1), (21, 1), (24, 1), (26, 2), (27, 1), (28, 1), (29, 1), (30, 1), (31, 1), (32, 1), (33, 1), (34, 1), (35, 1), (36, 1), (37, 1), (38, 1), (39, 1), (40, 1)], []]


In [19]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [20]:
from gensim import corpora
## create a toy corpus of 2 documents, as a plain Python list
# make one document empty, for the heck of it
corpora.MmCorpus.serialize('corpus.mm', corpus)

In [23]:
tfidf = models.TfidfModel(corpus)

In [24]:
print tfidf

TfidfModel(num_docs=76259, num_nnz=763697)


In [25]:
corpus_tfidf = tfidf[corpus]

In [26]:
for doc in corpus_tfidf[:5]:
    print(doc)

[(0, 0.30963556516992646), (1, 0.3027900247274372), (2, 0.5055098852604292), (3, 0.3066097953009965), (4, 0.30552519202221323), (5, 0.3729079611319987), (6, 0.48009188349845316)]
[(7, 0.5415783824864412), (8, 0.43997267230801995), (9, 0.7163217874988236)]
[(3, 0.22424505183390556), (4, 0.22345180607921197), (10, 0.2827344791118436), (11, 0.26012336206516457), (12, 0.13199840167827614), (13, 0.25330646628640713), (14, 0.2223638254462579), (15, 0.23981603930345458), (16, 0.3233341657460019), (17, 0.22696462560439337), (18, 0.1682829737109741), (19, 0.3168659291450756), (20, 0.16811800862602103), (21, 0.2094050202888492), (22, 0.13980664031755186), (23, 0.13606854509719263), (24, 0.24104006187605273), (25, 0.33253460828747355)]
[(3, 0.152852201717907), (4, 0.15231150144774852), (5, 0.18590339829556135), (11, 0.1773079418464696), (13, 0.17266133974692205), (15, 0.16346585717285717), (16, 0.2203943352181761), (17, 0.15470594535752688), (19, 0.21598538974714632), (21, 0.14273678790308994), (

In [29]:
lsi = models.LsiModel(corpus_tfidf, id2word=dictionary, num_topics=10) # initialize an LSI transformation
corpus_lsi = lsi[corpus_tfidf]

In [391]:
corpus_lda = lda[corpus_tfidf]

In [73]:
texts[0]

'united states securities and exchange commission washington, d.c'

In [209]:
for idx, doc in enumerate(corpus_lsi[100:105]):
    print doc, '#', texts[idx+100], '\n'

[(0, 0.0013806645717757303), (1, 0.003311636880689855), (2, 0.0018444115693276137), (3, 0.010499419469914241), (4, 0.04921707649113706), (5, -4.7001444755940728e-05), (6, 0.065265724924815111), (7, 0.0073514823861610505), (8, 0.029429086366544871), (9, -0.02498805525289614)] # the reports are also available on the sec’s website at www.sec.gov 

[(0, 0.00096373598947126161), (1, 0.0024177950164484007), (2, 0.0014784515272263194), (3, 0.0070016659220266137), (4, 0.035910634311202429), (5, -6.664626859603361e-05), (6, 0.040631874821222352), (7, 0.0038317877318656624), (8, 0.014748196238343397), (9, 0.025778494495059825)] # ________________________________________________________ * incorporated in delaware in 1926 as standard oil company of california, the company adopted the name chevron corporation in 1984 and chevrontexaco corporation in 2001 

[(0, 0.00031390023408125984), (1, 0.0011639291974074222), (2, 0.00049631892003702133), (3, 0.0032066483602865852), (4, 0.015701588449616957), (5

In [210]:
lsi = models.lsimodel.LsiModel(corpus_tfidf, id2word=dictionary, num_topics=400)

In [381]:
def similar_lsi(string, n_results):
    print string, '\n'
    vec_bow = dictionary.doc2bow(string.lower().split())
    vec_lsi = lsi[vec_bow] # convert the query to LSI space
    index, score = zip(*sorted(vec_lsi, key=lambda tup: tup[1], reverse=True))
    for i in range(n_results):
        print score[i], texts[index[i]], '\n',
    print index[:20]

In [395]:
similar_lsi('currency money oil drilling shares', 5)

currency money oil drilling shares 

0.406970094427 title of each class 
0.273466897159 consolidated companies 25,707 
0.25674452865 at december 31. 
0.234358344696 6,481 
0.22684215097 this page intentionally left blank table of contents item 
(11, 131, 116, 129, 34, 31, 229, 140, 6, 111, 41, 157, 51, 43, 135, 156, 145, 63, 130, 173)


In [248]:
lsi.print_topics(10)

[(0,
  u'0.976*"\u2014" + 0.212*"$" + 0.029*")" + 0.012*"total" + 0.008*"%" + 0.007*"net" + 0.007*"other" + 0.007*"income" + 0.004*"december" + 0.004*"assets"'),
 (1,
  u'0.954*"$" + -0.214*"\u2014" + 0.184*")" + 0.059*"-" + 0.023*"total" + 0.020*"income" + 0.019*"net" + 0.016*"december" + 0.015*"31," + 0.015*"2014"'),
 (2,
  u'0.997*"-" + -0.065*"$" + 0.018*")" + 0.011*"2014" + 0.010*"\u2014" + 0.008*"2013" + 0.007*"december" + 0.007*"1" + 0.007*"31," + 0.006*"gas"'),
 (3,
  u'0.948*")" + -0.195*"$" + 0.080*"2014" + 0.078*"december" + 0.077*"31," + 0.072*"2013" + 0.052*"ended" + 0.045*"year" + 0.044*"net" + 0.039*"our"'),
 (4,
  u'0.382*"2014" + 0.343*"2013" + 0.341*"december" + 0.341*"31," + -0.245*")" + 0.237*"ended" + 0.197*"year" + 0.152*"our" + 0.131*"million" + 0.124*"as"'),
 (5,
  u'-1.000*"\u2014." + 0.007*"$" + -0.002*")" + -0.001*"\u2014" + -0.000*"2014" + -0.000*"2013" + -0.000*"december" + -0.000*"31," + 0.000*"total" + -0.000*"ended"'),
 (6,
  u'-0.437*"2014" + -0.402*"20

In [398]:
for i in texts2[1231:1241]:
    print i,'\n'

['information', 'required', 'by', 'item', '201(d)', 'regulation', 's-k', 'contained', 'under', 'heading', '\xe2\x80\x9cequity', 'compensation', 'plan', 'information\xe2\x80\x9d', '2015', 'proxy', 'statement', 'is', 'incorporated', 'by', 'reference', 'into', 'this', 'annual', 'report', 'on', 'form', '10-k'] 

['item', '13'] 

['certain', 'relationships', 'related', 'transactions,', 'director', 'independence', 'information', 'required', 'by', 'item', '404', 'regulation', 's-k', 'contained', 'under', 'heading', '\xe2\x80\x9ccorporate', 'governance', '\xe2\x80\x94', 'related', 'person', 'transactions\xe2\x80\x9d', '2015', 'proxy', 'statement', 'is', 'incorporated', 'by', 'reference', 'into', 'this', 'annual', 'report', 'on', 'form', '10-k'] 

['information', 'required', 'by', 'item', '407(a)', 'regulation', 's-k', 'contained', 'under', 'heading', '\xe2\x80\x9ccorporate', 'governance', '\xe2\x80\x94', 'director', 'independence\xe2\x80\x9d', '2015', 'proxy', 'statement', 'is', 'incorporated'

In [239]:
lda = models.ldamodel.LdaModel(corpus_tfidf, id2word=dictionary, num_topics=400)

In [279]:
def similar_lda(string, n_results):
    vec_bow = dictionary.doc2bow(string.lower().split())
    vec_lda = lda[vec_bow] # convert the query to LSI space
#     index, score = zip(*sorted(vec_lda, key=lambda tup: tup[1], reverse=True))
#     for i in range(n_results):
#         print score[i], texts[index[i]], '\n'

In [297]:
vec_lda

[(75, 0.24742541081898647),
 (157, 0.12011891251241605),
 (173, 0.11138888888888814),
 (206, 0.15389856624005735),
 (210, 0.11138888888888816),
 (231, 0.035223777095203017),
 (376, 0.11138888888888814)]

In [303]:
lda = models.ldamodel.LdaModel(corpus_tfidf, id2word=dictionary, num_topics=400)

In [329]:
import gensim.similarities

In [392]:
index = gensim.similarities.MatrixSimilarity(lda[corpus_lda])

In [385]:
doc = 'the scores are weird'
vec_bow = dictionary.doc2bow(doc.lower().split())
vec_lda = lda[vec_bow] # convert the query to LSI space
# index, score = zip(*sorted(vec_lda, key=lambda tup: tup[1], reverse=True))
# for i in range(n_results):
#     print score[i], texts[index[i]], '\n',
# print index[:20]

In [386]:
ldasim = index[vec_lda]

In [387]:
ldaidx, ldascore = zip(*sorted(enumerate(ldasim), key=lambda item: item[1], reverse=True))

In [388]:
for i in range(10):
    print ldascore[i], texts2[i]

1.0 ['united', 'states', 'securities', 'exchange', 'commission', 'washington,', 'd.c']
1.0 ['20549', 'form', '10-k']
1.0 ['\xc3\xbe', 'annual', 'report', 'pursuant', 'section', '13', 'or', '15(d)', 'securities', 'exchange', 'act', '1934', 'fiscal', 'year', 'ended', 'december', '31,', '2014', 'or']
1.0 ['transition', 'report', 'pursuant', 'section', '13', 'or', '15(d)', 'securities', 'exchange', 'act', '1934', 'transition', 'period', 'from', 'commission', 'file', 'number', 'chevron', 'corporation', '(exact', 'name', 'registrant', 'as', 'specified', 'its', 'charter)', 'delaware']
1.0 []
1.0 ['6001', 'bollinger', 'canyon', 'road,', 'san', 'ramon,', 'california', '94583-2324', '(state', 'or', 'other', 'jurisdiction', 'incorporation', 'or', 'organization)']
1.0 ['(i.r.s']
1.0 ['employer', 'identification', 'no.)']
0.944221 ['(address', 'principal', 'executive', 'offices)', '(zip', 'code)']
0.910789 ['registrant\xe2\x80\x99s', 'telephone', 'number,', 'including', 'area', 'code']


In [376]:
lda.print_topics()

[(115,
  u'0.296*none + 0.126*staff + 0.107*unresolved + 0.092*comments + 0.000*evidence, + 0.000*2.58 + 0.000*$/mcfe + 0.000*8.1 + 0.000*conversions + 0.000*sizes'),
 (91,
  u'0.114*form + 0.100*reference + 0.083*report + 0.073*by + 0.053*incorporated + 0.053*on + 0.041*10-k + 0.031*this + 0.030*mississippi + 0.029*proxy'),
 (5,
  u'0.369*change + 0.248*2017 + 0.076*climate + 0.024*pages + 0.016*reached + 0.016*through + 0.014*east, + 0.011*build + 0.009*middle + 0.007*reduction,'),
 (59,
  u'0.155*127 + 0.055*by, + 0.000*\u2014 + 0.000*corporation + 0.000*moody\u2019s + 0.000*poor\u2019s + 0.000*investors + 0.000*of, + 0.000*guaranteed + 0.000*rated'),
 (233,
  u'0.235*impact + 0.145*performance + 0.070*field + 0.062*life + 0.039*apply + 0.034*least + 0.029*an + 0.027*utilize + 0.026*operating + 0.022*pressure'),
 (347,
  u'0.330*amortization + 0.103*reduced + 0.090*depletion + 0.090*depreciation, + 0.050*approximate + 0.048*llp + 0.019*31, + 0.018*million + 0.013*as + 0.012*by'),
 (

In [293]:
sims = index[vec_lda] # perform a similarity query against the corpus
print(list(enumerate(sims))) # print (document_number, document_similarity) 2-tuples
# [(0, 0.99809301), (1, 0.93748635), (2, 0.99844527), (3, 0.9865886), (4, 0.90755945),
# (5, -0.12416792), (6, -0.1063926), (7, -0.098794639), (8, 0.05004178)]

TypeError: tuple indices must be integers, not list

In [8]:
# cleansent = re.sub('[^A-Za-z]+', ' ', tmptmp)

In [9]:
# cleansent = cleansent.lower()

In [ ]:
doc = "Human computer interaction"








In [9]:
sentences = []
for i in cleansent.split(" "):
    if len(i)>3: sentences.append(i)
sentences = " ".join(sentences)

In [10]:
sentences[:400]

'united states securities exchange commission washington form annual report pursuant section securities exchange fiscal year ended december transition report pursuant section securities exchange transition period from commission file number chevron corporation exact name registrant specified charter delaware bollinger canyon road ramon california state other jurisdiction incorporation organization '

In [11]:
with open("multiple2014.txt", "wb") as text_file:
    text_file.write(sentences)

In [12]:
!head -c300 multiple2014.txt

united states securities exchange commission washington form annual report pursuant section securities exchange fiscal year ended december transition report pursuant section securities exchange transition period from commission file number chevron corporation exact name registrant specified charter 

In [16]:
import word2vec

In [41]:
word2vec.word2vec('multiple2014.txt', 'multiple_output.bin', size=10000, verbose=True, min_count=1)

Starting training using file multiple2014.txt
Vocab size: 10407
Words in train file: 614098
Alpha: 0.000785  Progress: 97.20%  Words/thread/sec: 4.87k  

In [42]:
model = word2vec.load('multiple_output.bin')

In [43]:
import pprint

In [44]:
model.vocab[:100]

array([u'</s>', u'that', u'with', u'million', u'december', u'from',
       u'other', u'company', u'financial', u'production', u'cash',
       u'operations', u'which', u'natural', u'year', u'have', u'income',
       u'costs', u'these', u'total', u'assets', u'ended', u'under',
       u'interest', u'value', u'reserves', u'operating', u'business',
       u'crude', u'prices', u'future', u'consolidated', u'this', u'will',
       u'statements', u'common', u'such', u'units', u'partnership',
       u'related', u'could', u'were', u'based', u'capital', u'table',
       u'price', u'sales', u'including', u'energy', u'wells', u'notes',
       u'during', u'stock', u'drilling', u'activities', u'properties',
       u'certain', u'proved', u'years', u'credit', u'market', u'general',
       u'results', u'loss', u'products', u'period', u'management', u'term',
       u'development', u'approximately', u'well', u'also', u'cost',
       u'agreement', u'fair', u'result', u'equity', u'increase',
       u'corpora

In [45]:
model.vectors.shape

(10407, 10000)

In [47]:
indexes, metrics = model.cosine('currency')
model.generate_response(indexes, metrics, clusters=True)

rec.array([(u'foreign', 0.9569433804752123), (u'translation', 0.9142850744878005),
 (u'derivatives', 0.8292544164541273), (u'selects', 0.7984495616861635),
 (u'reclassifies', 0.7839928287947682), (u'swaps', 0.7807568120132391),
 (u'unrealized', 0.772210304160807), (u'gains', 0.7692493403238411),
 (u'translated', 0.7609531334093015), (u'denominated', 0.7597829030433463)], 
          dtype=[(u'word', '<U78'), (u'metric', '<f8')])

In [51]:
indexes, metrics = model.cosine('petroleum')
model.generate_response(indexes, metrics, clusters=False)

rec.array([(u'tightens', 0.8446703502914263), (u'specialties', 0.8268779608433172),
 (u'edgewood', 0.7958180872455813), (u'exporting', 0.7822678247941249),
 (u'refiner', 0.7784209664413801), (u'gavilon', 0.7710303193068555),
 (u'petrochemical', 0.7708759116590465), (u'society', 0.7659609211095834),
 (u'wholesale', 0.7646051410376627), (u'polyolester', 0.7643957581323834)], 
          dtype=[(u'word', '<U78'), (u'metric', '<f8')])

In [25]:
def simwords(stub):
#     global indexes
    indexes, metrics = model.cosine(stub)
    return model.generate_response(indexes, metrics)

In [26]:
simwords('prices')

rec.array([(u'curves', 0.8247690123793273), (u'differentials', 0.7990578233007499),
 (u'pricing', 0.7961579855043138), (u'volatility', 0.7874875003611536),
 (u'decline', 0.7723233155824794), (u'prevailing', 0.7695081567241682),
 (u'price', 0.7687901914838995), (u'correlation', 0.7678445060097845),
 (u'declines', 0.7594943396894593), (u'movements', 0.7594299258921632)], 
          dtype=[(u'word', '<U78'), (u'metric', '<f8')])

In [27]:
simwords('chairman')

rec.array([(u'chair', 0.9693560487788974), (u'sutherland', 0.965843710764715),
 (u'wascom', 0.9656610493027291), (u'craig', 0.9540980343648446),
 (u'appointment', 0.9539335993662164), (u'mcgrath', 0.9534643136445273),
 (u'secretary', 0.9475133264971107), (u'served', 0.9473083337322769),
 (u'treasurer', 0.9460238403668957), (u'jonathan', 0.9448189644511172)], 
          dtype=[(u'word', '<U78'), (u'metric', '<f8')])

In [107]:
tmptmp = re.sub( '\s+', ' ', tmptmp ).strip()
tmptmp = tmptmp.replace('\n \n ', '. ')
tmptmp = tmptmp.replace("\n", " ", )
tmptmp = tmptmp.replace(" o ", ". ")

In [138]:
tmptmp[:1000].decode("UTF8")

u'10-K 1 cvx-123114x10kdoc.htm 10-K UNITED STATES SECURITIES AND EXCHANGE COMMISSION Washington, D.C. 20549 Form 10-K \xfe ANNUAL REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934 For the fiscal year ended December 31, 2014 OR. TRANSITION REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934 For the transition period from to Commission File Number 001-00368 Chevron Corporation (Exact name of registrant as specified in its charter) Delaware 94-0890210 6001 Bollinger Canyon Road, San Ramon, California 94583-2324 (State or other jurisdiction of incorporation or organization) (I.R.S. Employer Identification No.) (Address of principal executive offices) (Zip Code) Registrant\u2019s telephone number, including area code (925) 842-1000 Securities registered pursuant to Section 12 (b) of the Act: Title of Each Class Name of Each Exchange on Which Registered Common stock, par value $.75 per share New York Stock Exchange, Inc. Indicate by check

In [166]:
sentences = tmptmp.split('. ')
# for i in tmptmp:
#     try: [sentences.append(j) for j in i.split('. ')]
#     except: sentences.append(i)

In [167]:
sentences[:5]

['10-K 1 cvx-123114x10kdoc.htm 10-K UNITED STATES SECURITIES AND EXCHANGE COMMISSION Washington, D.C',
 '20549 Form 10-K \xc3\xbe ANNUAL REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934 For the fiscal year ended December 31, 2014 OR',
 'TRANSITION REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934 For the transition period from to Commission File Number 001-00368 Chevron Corporation (Exact name of registrant as specified in its charter) Delaware 94-0890210 6001 Bollinger Canyon Road, San Ramon, California 94583-2324 (State or other jurisdiction of incorporation or organization) (I.R.S',
 'Employer Identification No.) (Address of principal executive offices) (Zip Code) Registrant\xe2\x80\x99s telephone number, including area code (925) 842-1000 Securities registered pursuant to Section 12 (b) of the Act: Title of Each Class Name of Each Exchange on Which Registered Common stock, par value $.75 per share New York Stock Exchange, Inc

In [168]:
# for idx, i in enumerate(sentences): 
#     if len(i) > 500: print idx, i

In [179]:
counts = pd.Series([len(i) for i in sentences])#.plot(kind='kde', xlim=(0,500))#, bins=1000)

In [182]:
counts.describe()

count    2238.000000
mean      184.252904
std       235.246067
min         3.000000
25%        85.000000
50%       135.000000
75%       210.000000
max      3199.000000
dtype: float64

In [186]:
morethanten =[]
for i in sentences:
    if len(i)>10: morethanten.append(i)

In [187]:
#### print out the number of sentences in the corpus

In [188]:
len(morethanten)

2151

In [195]:
from pprint import pprint
pprint(sentences[:5])

['10-K 1 cvx-123114x10kdoc.htm 10-K UNITED STATES SECURITIES AND EXCHANGE COMMISSION Washington, D.C',
 '20549 Form 10-K \xc3\xbe ANNUAL REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934 For the fiscal year ended December 31, 2014 OR',
 'TRANSITION REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934 For the transition period from to Commission File Number 001-00368 Chevron Corporation (Exact name of registrant as specified in its charter) Delaware 94-0890210 6001 Bollinger Canyon Road, San Ramon, California 94583-2324 (State or other jurisdiction of incorporation or organization) (I.R.S',
 'Employer Identification No.) (Address of principal executive offices) (Zip Code) Registrant\xe2\x80\x99s telephone number, including area code (925) 842-1000 Securities registered pursuant to Section 12 (b) of the Act: Title of Each Class Name of Each Exchange on Which Registered Common stock, par value $.75 per share New York Stock Exchange, Inc

ValueError: invalid literal for int() with base 10: '10-K'

Starting training using file chevron 10K 2014

Vocab size (unigrams + bigrams): 24917
Words in train file: 64979


Exception: Could not identify kind

TypeError: accuracy() takes at least 2 arguments (1 given)